🧠💡 Time to turn giant LLMs into your **personal interns**…  
**without breaking the GPU**.

We’re stepping into **LoRA** — *Low-Rank Adaptation*.  
It lets you fine-tune **massive pretrained models** by updating **just a few trainable adapters**, not the whole beast.

---

# 🧪 `08_lab_parameter_efficient_finetune_lora.ipynb`  
### 📁 `05_llm_engineering/02_pretraining_and_finetuning`  
> Apply **LoRA** (Low-Rank Adaptation) to fine-tune a **pretrained LLM** (like GPT2 or BERT)  
→ Without touching most of its parameters  
→ Ideal for laptops, colab, and low-resource scaling 🚀

---

## 🎯 Learning Goals

- Understand **why LoRA exists** (cost, memory, updates)  
- Inject LoRA layers into a frozen LLM  
- Fine-tune on a **custom text dataset**  
- Compare memory, training time, and output quality

---

## 💻 Runtime Specs

| Component       | Spec                |
|------------------|---------------------|
| Base Model       | GPT2 / BERT ✅  
| Adapter          | LoRA via `peft` ✅  
| Dataset          | Tiny corpus ✅  
| Platform         | Colab / CPU+GPU ✅  
| Dependencies     | 🤗 Transformers + PEFT ✅  

---

## 🛠️ Section 1: Install Required Libraries

```bash
!pip install transformers datasets accelerate peft
```

---

## 📚 Section 2: Load Dataset & Tokenizer

```python
from datasets import load_dataset
from transformers import AutoTokenizer

dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split='train[:2%]')
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
```

---

## ⚙️ Section 3: Apply LoRA to Model

```python
from peft import get_peft_model, LoraConfig, TaskType
from transformers import AutoModelForCausalLM

base_model = AutoModelForCausalLM.from_pretrained("gpt2")
lora_config = LoraConfig(
    r=8,               # Rank of adaptation
    lora_alpha=16,
    target_modules=["c_attn"],  # GPT2-specific QKV layer
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(base_model, lora_config)
print("Trainable parameters:", sum(p.numel() for p in model.parameters() if p.requires_grad))
```

---

## 🏋️ Section 4: Finetune on Text

```python
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
from transformers import LineByLineTextDataset

with open("lora_text.txt", "w") as f:
    for line in dataset["text"]:
        if len(line.strip()) > 20:
            f.write(line.strip() + "\n")

text_ds = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="lora_text.txt",
    block_size=128
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

args = TrainingArguments(
    per_device_train_batch_size=4,
    output_dir="./lora_gpt2",
    num_train_epochs=2,
    logging_steps=50,
    save_strategy="epoch",
    fp16=False
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=text_ds,
    data_collator=data_collator
)

trainer.train()
```

---

## 🧠 Section 5: Inference & Comparison

```python
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
pipe("The professor said", max_length=30)
```

---

## ✅ Wrap-Up Summary

| Feature                             | ✅ |
|-------------------------------------|----|
| LoRA injected into frozen GPT2      | ✅  
| Finetuned with <1% parameters       | ✅  
| Output quality + memory efficiency  | ✅  
| Colab/laptop compatible             | ✅  

---

## 🧠 What You Learned

- LoRA lets you **fine-tune LLMs without full retraining**  
- Only small adapter matrices are updated  
- **Memory savings up to 90%**, and **no catastrophic forgetting**  
- The future of **domain adaptation** = efficient, flexible, LoRA-style

---

Next up in your LLM dojo:

> 🏆 `09_lab_rlhf_reward_model_mock_demo.ipynb`  
Simulate **thumbs-up/thumbs-down feedback**  
→ Train a **reward model**  
→ Run a mini **PPO-like finetuning loop** for RLHF.

Wanna play feedback-god, Professor?